# Quantum transport mini-projects (ASE + tight-binding + NEGF)

These notebooks are designed for **hands-on, live coding**: build a small graphene-based device, assemble a Hamiltonian, and compute bandstructure/DOS/transmission **without** QuantumATK/SIESTA.

**Core packages**: `numpy`, `scipy`, `matplotlib`, `ase`  
**Optional**: `sisl` (for structure I/O, Hamiltonian export, and extra analysis)

> Tip: Run on a laptop. Keep systems small (≤ a few hundred atoms) for fast feedback.

In [ ]:
# Colab / local: install essentials
# If you're local and already have these, you can comment this out.
!pip -q install ase scipy matplotlib

# Optional (uncomment if you want to use SISL utilities)
# !pip -q install sisl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple, Optional, List

from ase.build import graphene_nanoribbon
from ase.visualize import view

In [ ]:
import scipy.linalg as la

def fermi(E, mu=0.0, kBT=0.02585):
    """Fermi-Dirac distribution. kBT in eV."""
    x = (E - mu) / max(kBT, 1e-12)
    # numerically stable
    return np.where(x>50, 0.0, np.where(x<-50, 1.0, 1.0/(1.0+np.exp(x))))

def dfermi_dE(E, mu=0.0, kBT=0.02585):
    """Derivative -df/dE (positive)."""
    f = fermi(E, mu, kBT)
    return (f*(1-f))/max(kBT, 1e-12)

def sancho_rubio_surface_gf(E, H00, H01, eta=1e-6, maxiter=200, tol=1e-12):
    """Surface Green's function for a semi-infinite lead using Sancho-Rubio decimation.

    H00: onsite block (n x n)
    H01: coupling to next principal layer (n x n)
    """
    z = (E + 1j*eta)
    n = H00.shape[0]
    I = np.eye(n, dtype=complex)
    a = H01.copy().astype(complex)
    b = H01.conj().T.copy().astype(complex)
    e = H00.copy().astype(complex)
    g = la.inv(z*I - e)
    for _ in range(maxiter):
        ag = a @ g
        bg = b @ g
        e_new = e + ag @ b + bg @ a
        a_new = ag @ a
        b_new = bg @ b
        g_new = la.inv(z*I - e_new)
        if np.max(np.abs(g_new - g)) < tol:
            g = g_new
            break
        e, a, b, g = e_new, a_new, b_new, g_new
    return g

def self_energy(E, H00, H01, eta=1e-6):
    gs = sancho_rubio_surface_gf(E, H00, H01, eta=eta)
    return H01.conj().T @ gs @ H01

def transmission(E, Hc, SigmaL, SigmaR, eta=1e-6):
    n = Hc.shape[0]
    I = np.eye(n, dtype=complex)
    G = la.inv((E+1j*eta)*I - Hc - SigmaL - SigmaR)
    GL = 1j*(SigmaL - SigmaL.conj().T)
    GR = 1j*(SigmaR - SigmaR.conj().T)
    T = np.real(np.trace(GL @ G @ GR @ G.conj().T))
    return T, G, GL, GR

In [ ]:
def build_tb_from_positions(pos, cutoff=1.7, t=-2.7, onsite=0.0):
    """Build a simple nearest-neighbor TB Hamiltonian from atomic positions.
    Uses a distance cutoff to define neighbors.
    """
    pos = np.asarray(pos)
    n = len(pos)
    H = np.zeros((n, n), dtype=float)
    np.fill_diagonal(H, onsite)
    # brute force neighbor search (fine for small n)
    for i in range(n):
        for j in range(i+1, n):
            d = np.linalg.norm(pos[i]-pos[j])
            if d < cutoff:
                H[i, j] = t
                H[j, i] = t
    return H

def block_tridiagonal_device(H0, H1, ncells):
    """Create central Hamiltonian for ncells repeats of principal layer.
    H0: intra-layer (n x n), H1: inter-layer coupling (n x n)
    Returns Hc (n*ncells x n*ncells)
    """
    n = H0.shape[0]
    Hc = np.zeros((n*ncells, n*ncells), dtype=float)
    for c in range(ncells):
        sl = slice(c*n,(c+1)*n)
        Hc[sl, sl] = H0
        if c < ncells-1:
            sl2 = slice((c+1)*n,(c+2)*n)
            Hc[sl, sl2] = H1
            Hc[sl2, sl] = H1.T
    return Hc

def add_adatom_onsite(H, idx, delta=1.0):
    """Mimic an adatom by shifting onsite energy at a site."""
    H = H.copy()
    H[idx, idx] += delta
    return H

def add_anderson_disorder(H, W=0.5, rng=None):
    """Random onsite disorder in [-W/2, W/2]."""
    rng = np.random.default_rng() if rng is None else rng
    H = H.copy()
    H[np.diag_indices_from(H)] += rng.uniform(-W/2, W/2, size=H.shape[0])
    return H

# Project 1 — Width-dependent graphene nanoribbon (bands + transmission)

Goal: learn how the **Hamiltonian changes with width** and how that impacts **bandstructure and transmission**.

We will:
1. Build armchair graphene nanoribbons (AGNR) with ASE.
2. Construct a nearest-neighbor tight-binding Hamiltonian (π-orbital model).
3. Compute bandstructure (Bloch Hamiltonian) and a simple 2-probe transmission using NEGF.

In [ ]:
!pip -q install ase scipy matplotlib

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from ase.build import graphene_nanoribbon

## 1) Build a ribbon unit cell

In [ ]:
def make_agnr(width=7, vacuum=8.0):
    # type='armchair' gives armchair edges; length=1 => one unit along transport
    # periodic=True makes it periodic along the ribbon axis
    atoms = graphene_nanoribbon(width=width, length=1, type='armchair', saturated=False, vacuum=vacuum, periodic=True)
    return atoms

atoms = make_agnr(width=7)
print(atoms)
print("natoms:", len(atoms))
# view(atoms)  # uncomment in local Jupyter

## 2) Build principal-layer blocks H00 and H01

We create a **2-cell supercell** to detect couplings from cell 0 to cell 1.

In [ ]:
from ase.build import make_supercell

def principal_layer_blocks(atoms, cutoff=1.7, t=-2.7, onsite=0.0):
    # 2x along transport (assume transport is x)
    P = np.diag([2,1,1])
    at2 = make_supercell(atoms, P)
    n = len(atoms)
    # positions for cell0 and cell1
    p0 = atoms.get_positions()
    p2 = at2.get_positions()
    p1 = p2[n:2*n]  # second copy
    # intra-block from p0 only
    H00 = build_tb_from_positions(p0, cutoff=cutoff, t=t, onsite=onsite)
    # coupling between p0 and p1
    H01 = np.zeros((n,n), float)
    for i in range(n):
        for j in range(n):
            d = np.linalg.norm(p0[i]-p1[j])
            if d < cutoff:
                H01[i,j] = t
    return H00, H01

H00, H01 = principal_layer_blocks(atoms)
print("H00 shape:", H00.shape, "H01 shape:", H01.shape)

## 3) Bandstructure from Bloch Hamiltonian

For 1D periodic systems:

\[ H(k) = H_{00} + H_{01} e^{ik a} + H_{01}^T e^{-ik a} \]

In [ ]:
def bands_1d(H00, H01, a, nk=201):
    ks = np.linspace(-np.pi/a, np.pi/a, nk)
    evals = []
    for k in ks:
        Hk = H00 + H01*np.exp(1j*k*a) + H01.T*np.exp(-1j*k*a)
        w = np.linalg.eigvalsh(Hk)
        evals.append(np.sort(np.real(w)))
    return ks, np.array(evals)

a = atoms.cell.lengths()[0]
ks, E = bands_1d(H00.astype(complex), H01.astype(complex), a, nk=251)

plt.figure()
plt.plot(ks*a/np.pi, E, lw=0.8)
plt.xlabel(r"$k a / \pi$")
plt.ylabel("Energy (eV)")
plt.title("AGNR bandstructure (nearest-neighbor TB)")
plt.axhline(0, ls="--")
plt.show()

## 4) Transmission at E=0 vs ribbon width

For a ballistic ribbon, the number of open channels near the Fermi energy depends strongly on width.

In [ ]:
def device_transmission(H00, H01, ncells=6, Es=np.linspace(-1.5,1.5,401), eta=1e-5):
    n = H00.shape[0]
    Hc = block_tridiagonal_device(H00, H01, ncells).astype(complex)
    Ts = []
    for E in Es:
        sigL = self_energy(E, H00.astype(complex), H01.astype(complex), eta=eta)
        sigR = sigL  # identical leads
        SL = np.zeros_like(Hc); SR = np.zeros_like(Hc)
        SL[:n,:n] = sigL
        SR[-n:,-n:] = sigR
        T, *_ = transmission(E, Hc, SL, SR, eta=eta)
        Ts.append(T)
    return np.array(Ts)

widths = [5, 7, 9, 11]
Es = np.linspace(-1.5, 1.5, 501)

plt.figure()
for w in widths:
    at = make_agnr(width=w)
    H00w, H01w = principal_layer_blocks(at)
    Tw = device_transmission(H00w, H01w, ncells=6, Es=Es)
    plt.plot(Es, Tw, label=f"width={w} (natoms/cell={len(at)})")
plt.xlabel("Energy (eV)")
plt.ylabel("Transmission T(E)")
plt.title("Width-dependent transmission (AGNR, ballistic TB)")
plt.legend()
plt.show()

## Exercises

1. Change from **armchair** to **zigzag** ribbons (`type='zigzag'`) and compare bands/transmission.
2. Add an **edge potential**: shift onsite energy of atoms with largest |y| and see how a gap opens/closes.
3. Replace distance-based neighbor detection with a **graph-based neighbor list** (speedup for larger systems).